In [1]:
import numpy as np

In [4]:
a = np.random.random((3,3))
print(a[1,1])
print(a[1][1])

0.5016575227872817
0.5016575227872817


In [5]:
a.row

AttributeError: 'numpy.ndarray' object has no attribute 'row'

In [17]:
x = np.array([[1,1],[1,2],[2,2],[2,3]])
x.dot([1,2])

array([3, 5, 6, 8])

In [12]:
from scipy.sparse import coo_matrix
b = coo_matrix(a)

In [13]:
b.row

array([0, 0, 0, 1, 1, 1, 2, 2, 2], dtype=int32)

In [14]:
b.col

array([0, 1, 2, 0, 1, 2, 0, 1, 2], dtype=int32)

In [15]:
b.data

array([0.62101433, 0.02921091, 0.76046985, 0.86688877, 0.50165752,
       0.74683486, 0.01129112, 0.81042626, 0.25285452])

In [18]:
np.power(2,3)

8